In [8]:
import tweepy
import re
import csv
import pandas as pd
import matplotlib.pyplot as plt
import requests
import json
from datetime import datetime
import time

In [13]:
def get_blocknumber_by_timestamp(date_string):
    
    # Function retrieving block number by date in format YYY    
    
    # convert date string to timestamp
    date_object = datetime.strptime(date_string, '%Y-%m-%d %H:%M')
    timestamp = int(time.mktime(date_object.timetuple()))
    
    # Find closes block to give timestamp (date_string)
    # Set the API endpoint
    url = 'https://api.etherscan.io/api'

    # Set the API parameters
    params = {
        'module': 'block',
        'action': 'getblocknobytime',
        'timestamp': timestamp,
        'closest': 'before', # the closest available block to the provided timestamp, either BEFORE or AFTER - https://docs.etherscan.io/api-endpoints/blocks#get-block-number-by-timestamp
        'apikey': 'H7X31IMU7EMEEBDE4633U8F73SNMYDH5AZ'
    }

    # Send a GET request to the API endpoint with the defined parameters
    response = requests.get(url, params=params)

    # Parse the response JSON data
    data = response.json()['result']

    return data

get_blocknumber_by_timestamp("2023-03-10 12:00")

'16797293'

In [ ]:
def get_token_transactions_by_dates(token_address, start_date, end_date):
    
    # Set the API endpoint
    url = 'https://api.etherscan.io/api'
    
    # Set the API parameters
    params = {
        'module': 'account',
        'action': 'txlist',
        'address': token_address,
        'startblock': get_blocknumber_by_timestamp(start_date),
        'endblock': get_blocknumber_by_timestamp(end_date),
        'sort': 'desc',
        'apikey': 'H7X31IMU7EMEEBDE4633U8F73SNMYDH5AZ'
    }
    
    # Send a GET request to the API endpoint with the defined parameters
    response = requests.get(url, params=params)

    # Parse the response JSON data
    data = response.json()

    # Extract the transaction data from the response
    transactions = data['result']
    
    return transactions

x = get_token_transactions_by_dates("0xb7DcaBe6DA1194507522b553950cCB5A8060A439","2021-12-09 12:36","2023-03-12 22:40")

In [23]:
pd.DataFrame(x).head()[['blockNumber','timeStamp','from','to','value']]

,blockNumber,timeStamp,from,to,value
0,16714175,1677435959,0xb7dcabe6da1194507522b553950ccb5a8060a439,0xd4307e0acd12cf46fd6cf93bc264f5d5d1598792,777000000000000
1,15001552,1655803025,0xb7dcabe6da1194507522b553950ccb5a8060a439,0xe592427a0aece92de3edee1f18e0157c05861564,0
2,15001538,1655802849,0xb7dcabe6da1194507522b553950ccb5a8060a439,0xe592427a0aece92de3edee1f18e0157c05861564,0
3,14088465,1643297137,0xb7dcabe6da1194507522b553950ccb5a8060a439,0xdac17f958d2ee523a2206206994597c13d831ec7,0
4,14081397,1643201999,0xb7dcabe6da1194507522b553950ccb5a8060a439,0xb8901acb165ed027e32754e0ffe830802919727f,2005000000000000000
